In [ ]:
import h5py
import numpy as np
from ctapipe.tools.process import ProcessorTool
import yaml
from traitlets.config import Config

# Processor Tool set up and run

In [ ]:
filename = '/Users/vdk/muons2024/simtel_files/2024year_tuned_nooulier_reflectivity_additional/run101_muon.simtel.gz'

yaml_file_path = '/Users/vdk/Software/ctasoft/calibpipe/doc/source/examples/throughput/configurations/processor_tool_muon_configuration.yaml'

def dict_to_config(d):
    """Recursively convert a dictionary into a Config object."""
    config = Config()
    for key, value in d.items():
        # If the key starts with an uppercase letter, ensure the value is a Config instance
        if key[0].isupper() and isinstance(value, dict):
            config[key] = dict_to_config(value)
        else:
            config[key] = value
    return config

def load_config_from_yaml(yaml_file_path):
    # Load the YAML file
    with open(yaml_file_path, 'r') as file:
        yaml_data = yaml.safe_load(file)
    
    # Convert the YAML data (a dictionary) to a Config object
    config = dict_to_config(yaml_data)
    return config

# Example usage:

config = load_config_from_yaml(yaml_file_path)

config['DataWriter']['output_path'] = '/Users/vdk/muons2024/calibpipe/lstchain_ctapipe_migration_data/test_sim.h5'
config['DataWriter']['write_muon_parameters'] = True
config['EventSource']['input_url'] = '/Users/vdk/muons2024/simtel_files/2024year_tuned_nooulier_reflectivity_additional/run101_muon.simtel.gz'
config['EventSource']['max_events'] = 50
config['GlobalPeakWindowSum']['window_shift'] = 4
config['GlobalPeakWindowSum']['window_width'] = 8

In [ ]:
test_tool = ProcessorTool(config=config)

In [ ]:
test_tool.run()

# Processing .h5 file

In [ ]:
with h5py.File(config['DataWriter']['output_path'], "r") as h5_file:
    # Access a dataset
    dataset = h5_file["dl1/event/telescope/muon/tel_001"]
    test_dataset = h5_file.get("dl1/event/telescope/muon/tel_001")
    trigger_dataset = h5_file["dl1/event/subarray/trigger"]
    tigger_info = h5_file["dl1/event/telescope/trigger"]
    time_dataset = trigger_dataset["time"]
    trigger_lookup = trigger_dataset[:50]
    simulation_info = h5_file['configuration/simulation/run']
    print(type(simulation_info))
    simulation_dataset = simulation_info[()]
    
    # Read the first and last values
    first_time = time_dataset[0]
    last_time = time_dataset[-1]
    # Check the shape and data type of the dataset
    print("Dataset shape:", dataset.shape)
    print("Dataset dtype:", dataset.dtype)
    dataset_dtype = dataset.dtype
    trigger_dataset_dtype = trigger_dataset.dtype
    trigger_info_dtype = tigger_info.dtype
    simulation_dtype = simulation_info.dtype
    # Load the data from the dataset
    data = dataset[:50]
    data_trigger = trigger_dataset[:50]
    data_trigger_info = tigger_info[:50]
    #print("Data:", data)

# Simulation info

In [ ]:
for dtype_name in simulation_dtype.names:
    print(f"Field {dtype_name}: {simulation_dataset[dtype_name]}")